# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# Create a path to the csv 
schools_csv = "Resources/schools_complete.csv"
students_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

# Rename column header school_name to school
schools_df.rename(columns = {'school_name': 'school'}, inplace = True)
students_df.rename(columns = {'school_name': 'school'}, inplace = True)

merged_df = students_df.merge(schools_df, how = 'left', on = 'school')
merged_df.head()


,Student ID,student_name,gender,grade,school,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [91]:
# Calculate the total number of schools
school_count = len(merged_df['school'].unique())

# Calculate the total number of students
student_count = len(merged_df['Student ID'].unique())

# Calculate the total budget
budget = schools_df['budget'].sum()

# Calculate the average math score
avg_math_score = students_df['math_score'].mean()

# Calculate the average reading score
avg_reading_score = students_df['reading_score'].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_pass_rate = (avg_math_score + avg_reading_score) / 2

# Calculate the percentage of students with a passing math score (70 or greater)
count_passing_math = students_df.loc[students_df['math_score']>= 70]['math_score'].count()
percent_passing_math = (count_passing_math / student_count)


# Calculate the percentage of students with a passing reading score (70 or greater)
count_passing_reading = students_df.loc[students_df['reading_score']>= 70]['reading_score'].count()
percent_passing_reading = (count_passing_reading / student_count)


# Create a dataframe to hold the above results
district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[percent_passing_reading],
    "% Passing Math": [percent_passing_math],
    "Overall Passing Rate": [overall_pass_rate]

})

# Create a high level snapshot (in table form) of the district's key metrics, including:




# Optional: give the displayed data cleaner formatting
district_summary.style.format({"Total Budget": "${:,.2f}", 
                       "Total Students": "{:,}",        
                       "Average Reading Score": "{:,.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})




,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,"39,170","$24,649,428.00",81.9,79.0,85.8%,75.0%,8043.2%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [128]:
# Groups by school
school_group = merged_df.set_index('school').groupby(['school'])

# Groups by school type
school_type = schools_df.set_index('school')['type']

# Calculate total of students at each school
students_at_school = school_group['Student ID'].count()

# Calculate the school budget
school_budget = schools_df.set_index('school')['budget']

# Calculate School budget per student
budget_per_student = schools_df.set_index('school')['budget']/schools_df.set_index('school')['size']

# Calculate Average Math Score at each school
school_avg_math = round(school_group['math_score'].mean(),2)

# Calculate Average Reading Score at each school
school_avg_reading = round(school_group['reading_score'].mean(),2)

# Calculate the % of passing Math Scores at each school
school_percent_math = round(merged_df[merged_df['math_score'] >= 70].groupby('school')['Student ID'].count()/students_at_school,2)


# Calculate the % of passing Reading Scores at each school
school_percent_reading = round(merged_df[merged_df['reading_score'] >= 70].groupby('school')['Student ID'].count()/students_at_school,2)

# Calculate the overall percent of pass rate at each school
overall = round(merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('school')['Student ID'].count()/students_at_school,2)

# School summary
school_summary = pd.DataFrame({
    "School Type": school_type,
    "Total Students": students_at_school,
    "Per Student Budget": budget_per_student,
    "Total School Budget": school_budget,
    "Average Math Score": school_avg_math,
    "Average Reading Score": school_avg_reading,
    '% Passing Math': school_percent_math,
    '% Passing Reading': school_percent_reading,
    "Overall Passing Rate": overall
})



school_summary.style.format({"Total Students": "{:,}", 
                       "Per Student Budget": "${:,.2f}",
                       "Total School Budget": "${:,.2f}",           
                       "Average Reading Score": "{:,.2f}", 
                       "Average Math Score": "{:.2f}", 
                       "% Passing Math": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "Overall Passing Rate": "{:.1%}"})

school_summary


# Sort and Display the Top 5 schools by passing rate
top_5_schools = school_summary.sort_values("Overall Passing Rate", ascending = False)
top_5_schools.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858",$582,"$1,081,356",83.06,83.98,94.0%,97.0%,91.0%
Griffin High School,Charter,"1,468",$625,"$917,500",83.35,83.82,93.0%,97.0%,91.0%
Pena High School,Charter,962,$609,"$585,858",83.84,84.04,95.0%,96.0%,91.0%
Thomas High School,Charter,"1,635",$638,"$1,043,130",83.42,83.85,93.0%,97.0%,91.0%
Wilson High School,Charter,"2,283",$578,"$1,319,574",83.27,83.99,94.0%,97.0%,91.0%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [130]:
# Sort and Display the bottom 5 schools by passing rate
bottom_5_schools = top_5_schools.tail()
bottom_5_schools = school_summary.sort_values("Overall Passing Rate")
bottom_5_schools.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Figueroa High School,District,"2,949",$639,"$1,884,411",76.71,81.16,66.0%,81.0%,53.0%
Rodriguez High School,District,"3,999",$637,"$2,547,363",76.84,80.74,66.0%,80.0%,53.0%
Ford High School,District,"2,739",$644,"$1,763,916",77.1,80.75,68.0%,79.0%,54.0%
Hernandez High School,District,"4,635",$652,"$3,022,020",77.29,80.93,67.0%,81.0%,54.0%
Huang High School,District,"2,917",$655,"$1,910,635",76.63,81.18,66.0%,81.0%,54.0%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [131]:
# Creates table of average math scores by grade level for each school 
math_9 = students_df.loc[students_df['grade'] == '9th'].groupby('school')["math_score"].mean()
math_10 = students_df.loc[students_df['grade'] == '10th'].groupby('school')["math_score"].mean()
math_11 = students_df.loc[students_df['grade'] == '11th'].groupby('school')["math_score"].mean()
math_12 = students_df.loc[students_df['grade'] == '12th'].groupby('school')["math_score"].mean()

# Creates a data fram for the above info
math_scores = pd.DataFrame({
        "9th": math_9,
        "10th": math_10,
        "11th": math_11,
        "12th": math_12
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

# Format and display
math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [132]:
# Creates table of average math scores by grade level for each school 
reading_9 = students_df.loc[students_df['grade'] == '9th'].groupby('school')["reading_score"].mean()
reading_10 = students_df.loc[students_df['grade'] == '10th'].groupby('school')["reading_score"].mean()
reading_11 = students_df.loc[students_df['grade'] == '11th'].groupby('school')["reading_score"].mean()
reading_12 = students_df.loc[students_df['grade'] == '12th'].groupby('school')["reading_score"].mean()

# Creates a data fram for the above info
reading_scores = pd.DataFrame({
        "9th": reading_9,
        "10th": reading_10,
        "11th": reading_11,
        "12th": reading_12
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

# Format and display
reading_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [137]:
# Bins to sort Scores by School Spending
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

merged_df['spending_bins'] = pd.cut(merged_df['budget']/merged_df['size'], spending_bins, labels = group_names)

In [151]:
# Group by spending
spending_groups = merged_df.groupby('spending_bins')

# Calculates scores by school spending
avg_math_spend = spending_groups['math_score'].mean()
avg_reading_spend = spending_groups['reading_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending_groups['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending_groups['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/spending_groups['Student ID'].count()

# Create a data frame for the above info
scores_by_spending = pd.DataFrame({
    "Average Math Score": avg_math_spend,
    "Average Reading Score": avg_reading_spend,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
    
})

# Reordering columns per instructions layout
scores_by_spending = scores_by_spending[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spending.index.name = "Per Student Budget"
scores_by_spending = scores_by_spending.reindex(group_names)

#formating
scores_by_spending.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
<$585,83.4,84.0,93.7%,96.7%,90.6%
$585-615,83.5,83.8,94.1%,95.9%,90.1%
$615-645,78.1,81.4,71.4%,83.6%,60.3%
$645-675,77.0,81.0,66.2%,81.1%,53.5%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [162]:
# Create bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
merged_df['size_bins'] = pd.cut(merged_df['size'], size_bins, labels = group_names)


,Student ID,student_name,gender,grade,school,reading_score,math_score,School ID,type,size,budget,spending_bins,size_bins
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,$645-675,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,$645-675,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,$645-675,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,$645-675,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,$645-675,Large (2000-5000)
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635,$645-675,Large (2000-5000)
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635,$645-675,Large (2000-5000)
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635,$645-675,Large (2000-5000)
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635,$645-675,Large (2000-5000)
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635,$645-675,Large (2000-5000)


In [183]:
# Group by school size
school_size = merged_df.groupby("size_bins")

# Calculates scores by school size 
avg_math = school_size['math_score'].mean()
avg_reading = school_size['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()
pass_reading = merged_df[merged_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/school_size['Student ID'].count()

# Create a data frame for the above info
scores_by_school_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_reading,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_reading,
    "Overall Passing Rate": overall
})

# moving columns
scores_by_school_size.index.name = "Total Students"
scores_by_school_size = scores_by_school_size.reindex(group_names)
scores_by_school_size = scores_by_school_size.reindex(group_names)

# Format and Display
scores_by_school_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.8,83.8,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.4,93.6%,96.8%,90.6%
Large (2000-5000),77.5,77.5,68.7%,82.1%,56.6%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [185]:
# Sort by group by type of school
school_group = merged_df.groupby("type")


#calculations 
avg_math = school_group['math_score'].mean()
avg_reading = school_group['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('type')['Student ID'].count()/school_group['Student ID'].count()
pass_reading = merged_df[merged_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/school_group['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/school_group['Student ID'].count()

# Create a dataframe for the info above.           
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_reading,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_reading,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
school_group.index.name = "Type of School"


#formating
school_group.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

AttributeError: Cannot access attribute 'index' of 'DataFrameGroupBy' objects, try using the 'apply' method